In [1]:
import sqlite3
import pandas as pd
import os

print("🔧 CREATING PRODUCTION-LIKE DATABASE STRUCTURE")
print("="*70)

# Загружаем очищенные данные
df_clean = pd.read_csv('lending_club_final.csv')
print(f"✅ Loaded data: {len(df_clean):,} loans")

# Создаем SQLite базу данных
conn = sqlite3.connect('lending_club.db')
cursor = conn.cursor()

print("\n📊 Creating normalized database structure...")
print("="*70)

# ============================================
# ТАБЛИЦА 1: dim_borrowers (информация о заемщиках)
# ============================================
print("\n1️⃣ Creating 'dim_borrowers' table...")

borrowers = df_clean[['id', 'annual_inc', 'emp_length', 'home_ownership', 
                       'fico_range_low', 'fico_range_high', 'dti', 
                       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 
                       'pub_rec', 'revol_bal', 'revol_util', 'total_acc']].copy()

borrowers.columns = ['loan_id', 'annual_income', 'employment_length', 'home_ownership',
                     'fico_low', 'fico_high', 'debt_to_income', 'delinquencies_2yrs', 
                     'inquiries_6mths', 'open_accounts', 'public_records',
                     'revolving_balance', 'revolving_utilization', 'total_accounts']

borrowers.to_sql('dim_borrowers', conn, if_exists='replace', index=False)
print(f"   ✅ Created with {len(borrowers):,} records")

# ============================================
# ТАБЛИЦА 2: dim_loan_products (характеристики кредитов)
# ============================================
print("\n2️⃣ Creating 'dim_loan_products' table...")

products = df_clean[['id', 'loan_amnt', 'funded_amnt', 'term', 'int_rate', 
                      'installment', 'grade', 'sub_grade', 'purpose']].copy()

products.columns = ['loan_id', 'loan_amount', 'funded_amount', 'term', 'interest_rate',
                    'monthly_installment', 'grade', 'sub_grade', 'purpose']

products.to_sql('dim_loan_products', conn, if_exists='replace', index=False)
print(f"   ✅ Created with {len(products):,} records")

# ============================================
# ТАБЛИЦА 3: dim_geography (географические данные)
# ============================================
print("\n3️⃣ Creating 'dim_geography' table...")

geography = df_clean[['id', 'addr_state', 'zip_code']].copy()
geography.columns = ['loan_id', 'state', 'zip_code']

geography.to_sql('dim_geography', conn, if_exists='replace', index=False)
print(f"   ✅ Created with {len(geography):,} records")

# ============================================
# ТАБЛИЦА 4: fact_loan_performance (результаты/статус)
# ============================================
print("\n4️⃣ Creating 'fact_loan_performance' table...")

performance = df_clean[['id', 'loan_status', 'loan_outcome', 
                         'issue_d', 'issue_date', 'issue_year']].copy()

performance.columns = ['loan_id', 'loan_status', 'outcome', 
                       'issue_date_str', 'issue_date', 'issue_year']

performance.to_sql('fact_loan_performance', conn, if_exists='replace', index=False)
print(f"   ✅ Created with {len(performance):,} records")

# ============================================
# СОЗДАНИЕ ИНДЕКСОВ для оптимизации
# ============================================
print("\n5️⃣ Creating indexes for query optimization...")

indexes = [
    "CREATE INDEX IF NOT EXISTS idx_borrowers_loan_id ON dim_borrowers(loan_id)",
    "CREATE INDEX IF NOT EXISTS idx_products_loan_id ON dim_loan_products(loan_id)",
    "CREATE INDEX IF NOT EXISTS idx_geography_loan_id ON dim_geography(loan_id)",
    "CREATE INDEX IF NOT EXISTS idx_performance_loan_id ON fact_loan_performance(loan_id)",
    "CREATE INDEX IF NOT EXISTS idx_performance_outcome ON fact_loan_performance(outcome)",
    "CREATE INDEX IF NOT EXISTS idx_products_grade ON dim_loan_products(grade)",
    "CREATE INDEX IF NOT EXISTS idx_geography_state ON dim_geography(state)",
]

for idx_query in indexes:
    cursor.execute(idx_query)
    
conn.commit()
print("   ✅ Created 7 indexes")

# ============================================
# ПРОВЕРКА СОЗДАННЫХ ТАБЛИЦ
# ============================================
print("\n" + "="*70)
print("📊 DATABASE SUMMARY:")
print("="*70)

cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()

for table in tables:
    table_name = table[0]
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    count = cursor.fetchone()[0]
    print(f"   ✅ {table_name}: {count:,} records")

print("\n" + "="*70)
print("✅ DATABASE CREATED SUCCESSFULLY!")
print("="*70)
print(f"\nDatabase file: {os.path.abspath('lending_club.db')}")
print(f"Size: {os.path.getsize('lending_club.db') / (1024**2):.2f} MB")

conn.close()

🔧 CREATING PRODUCTION-LIKE DATABASE STRUCTURE
✅ Loaded data: 1,347,721 loans

📊 Creating normalized database structure...

1️⃣ Creating 'dim_borrowers' table...
   ✅ Created with 1,347,721 records

2️⃣ Creating 'dim_loan_products' table...
   ✅ Created with 1,347,721 records

3️⃣ Creating 'dim_geography' table...
   ✅ Created with 1,347,721 records

4️⃣ Creating 'fact_loan_performance' table...
   ✅ Created with 1,347,721 records

5️⃣ Creating indexes for query optimization...
   ✅ Created 7 indexes

📊 DATABASE SUMMARY:
   ✅ dim_borrowers: 1,347,721 records
   ✅ dim_loan_products: 1,347,721 records
   ✅ dim_geography: 1,347,721 records
   ✅ fact_loan_performance: 1,347,721 records

✅ DATABASE CREATED SUCCESSFULLY!

Database file: C:\Users\User\lending_club.db
Size: 375.04 MB


In [5]:
import sqlite3
import pandas as pd

print("\n" + "="*80)
print("📊 SQL QUERY 1: CUSTOM RISK SCORING MODEL")
print("="*80)

# Подключаемся к БД
conn = sqlite3.connect('lending_club.db')

# СЛОЖНЫЙ SQL ЗАПРОС с CTE и CASE WHEN
query1 = """
WITH risk_factors AS (
    -- Шаг 1: Рассчитываем risk score для каждого фактора
    SELECT 
        perf.loan_id,
        p.grade,
        p.term,
        p.loan_amount,
        p.interest_rate,
        b.annual_income,
        b.debt_to_income,
        b.fico_low,
        perf.outcome,
        
        -- Risk Score Components (1 = низкий риск, 4 = высокий риск)
        CASE 
            WHEN p.grade IN ('A', 'B') THEN 1
            WHEN p.grade IN ('C', 'D') THEN 2
            WHEN p.grade IN ('E', 'F') THEN 3
            ELSE 4
        END as grade_risk_score,
        
        CASE 
    WHEN b.debt_to_income < 10 THEN 1   -- Более строго!
    WHEN b.debt_to_income < 20 THEN 2
    WHEN b.debt_to_income < 30 THEN 3
    ELSE 4
END as dti_risk_score,        
        CASE 
            WHEN b.fico_low >= 750 THEN 1
            WHEN b.fico_low >= 700 THEN 2
            WHEN b.fico_low >= 650 THEN 3
            ELSE 4
        END as fico_risk_score,
        
        CASE 
            WHEN p.term = ' 36 months' THEN 1
            ELSE 2
        END as term_risk_score
        
    FROM fact_loan_performance perf
    INNER JOIN dim_loan_products p ON perf.loan_id = p.loan_id
    INNER JOIN dim_borrowers b ON perf.loan_id = b.loan_id
    WHERE perf.outcome IN ('Good', 'Bad')
)
-- Шаг 2: Агрегируем по composite risk score
SELECT 
    (grade_risk_score + dti_risk_score + fico_risk_score + term_risk_score) as composite_risk_score,
    COUNT(*) as loan_count,
    ROUND(100.0 * SUM(CASE WHEN outcome = 'Bad' THEN 1 ELSE 0 END) / COUNT(*), 2) as default_rate,
    ROUND(AVG(loan_amount), 0) as avg_loan_amount,
    ROUND(AVG(interest_rate), 2) as avg_interest_rate,
    ROUND(AVG(debt_to_income), 2) as avg_dti,
    ROUND(AVG(fico_low), 0) as avg_fico
FROM risk_factors
GROUP BY composite_risk_score
ORDER BY composite_risk_score;
"""

# Выполняем запрос
result1 = pd.read_sql_query(query1, conn)

print("\n🎯 Results: Risk Score vs Default Rate")
print("-"*80)
print(result1.to_string(index=False))

print("\n💡 Key Insights:")
print(f"   • Lowest risk score ({result1['composite_risk_score'].min()}): {result1.iloc[0]['default_rate']:.2f}% default rate")
print(f"   • Highest risk score ({result1['composite_risk_score'].max()}): {result1.iloc[-1]['default_rate']:.2f}% default rate")
print(f"   • Risk correlation: {result1.iloc[-1]['default_rate'] / result1.iloc[0]['default_rate']:.1f}x higher for max vs min score")

conn.close()

print("\n" + "="*80)
print("✅ QUERY 1 COMPLETED!")
print("="*80)


📊 SQL QUERY 1: CUSTOM RISK SCORING MODEL

🎯 Results: Risk Score vs Default Rate
--------------------------------------------------------------------------------
 composite_risk_score  loan_count  default_rate  avg_loan_amount  avg_interest_rate  avg_dti  avg_fico
                    4       24823          4.83          13296.0               7.24     5.64     778.0
                    5       83109          6.76          13733.0               8.31     9.64     744.0
                    6      203740          9.48          13621.0               9.54    12.61     713.0
                    7      305386         14.18          13323.0              11.35    15.70     695.0
                    8      308005         20.16          13818.0              13.70    18.84     687.0
                    9      231873         27.36          15130.0              15.80    22.57     683.0
                   10      127638         34.77          16642.0              18.05    26.56     680.0
              

In [6]:
import sqlite3
import pandas as pd

print("\n" + "="*80)
print("👀 EXPLORING DATABASE TABLES")
print("="*80)

# Подключаемся к БД
conn = sqlite3.connect('lending_club.db')

# ============================================
# ТАБЛИЦА 1: dim_borrowers
# ============================================
print("\n📊 TABLE 1: dim_borrowers (Информация о заемщиках)")
print("-"*80)

query_borrowers = """
SELECT * FROM dim_borrowers LIMIT 10;
"""

df_borrowers = pd.read_sql_query(query_borrowers, conn)
print(df_borrowers)

print(f"\n   Total records: {pd.read_sql_query('SELECT COUNT(*) as cnt FROM dim_borrowers', conn)['cnt'][0]:,}")
print(f"   Columns: {len(df_borrowers.columns)}")

# ============================================
# ТАБЛИЦА 2: dim_loan_products
# ============================================
print("\n\n📊 TABLE 2: dim_loan_products (Характеристики кредитов)")
print("-"*80)

query_products = """
SELECT * FROM dim_loan_products LIMIT 10;
"""

df_products = pd.read_sql_query(query_products, conn)
print(df_products)

print(f"\n   Total records: {pd.read_sql_query('SELECT COUNT(*) as cnt FROM dim_loan_products', conn)['cnt'][0]:,}")
print(f"   Columns: {len(df_products.columns)}")

# ============================================
# ТАБЛИЦА 3: dim_geography
# ============================================
print("\n\n📊 TABLE 3: dim_geography (География)")
print("-"*80)

query_geography = """
SELECT * FROM dim_geography LIMIT 10;
"""

df_geography = pd.read_sql_query(query_geography, conn)
print(df_geography)

print(f"\n   Total records: {pd.read_sql_query('SELECT COUNT(*) as cnt FROM dim_geography', conn)['cnt'][0]:,}")
print(f"   Columns: {len(df_geography.columns)}")

# ============================================
# ТАБЛИЦА 4: fact_loan_performance
# ============================================
print("\n\n📊 TABLE 4: fact_loan_performance (Результаты кредитов)")
print("-"*80)

query_performance = """
SELECT * FROM fact_loan_performance LIMIT 10;
"""

df_performance = pd.read_sql_query(query_performance, conn)
print(df_performance)

print(f"\n   Total records: {pd.read_sql_query('SELECT COUNT(*) as cnt FROM fact_loan_performance', conn)['cnt'][0]:,}")
print(f"   Columns: {len(df_performance.columns)}")

# ============================================
# ПРИМЕР JOIN 
# ============================================
print("\n\n" + "="*80)
print("🔗 EXAMPLE: HOW TABLES ARE CONNECTED (JOIN)")
print("="*80)

query_join = """
SELECT 
    perf.loan_id,
    perf.outcome,
    p.grade,
    p.loan_amount,
    b.annual_income,
    b.fico_low,
    g.state
FROM fact_loan_performance perf
INNER JOIN dim_loan_products p ON perf.loan_id = p.loan_id
INNER JOIN dim_borrowers b ON perf.loan_id = b.loan_id
INNER JOIN dim_geography g ON perf.loan_id = g.loan_id
LIMIT 10;
"""

df_join = pd.read_sql_query(query_join, conn)
print("\nПример объединения всех 4 таблиц:")
print(df_join)

# ============================================
# СТАТИСТИКА ПО КАЖДОЙ ТАБЛИЦЕ
# ============================================
print("\n\n" + "="*80)
print("📈 TABLE STATISTICS")
print("="*80)

# dim_borrowers - распределение по FICO
print("\n1️⃣ dim_borrowers - FICO Score Distribution:")
fico_dist = pd.read_sql_query("""
    SELECT 
        CASE 
            WHEN fico_low < 650 THEN '600-650'
            WHEN fico_low < 700 THEN '650-700'
            WHEN fico_low < 750 THEN '700-750'
            ELSE '750+'
        END as fico_range,
        COUNT(*) as count
    FROM dim_borrowers
    GROUP BY fico_range
    ORDER BY fico_range;
""", conn)
print(fico_dist)

# dim_loan_products - распределение по Grade
print("\n2️⃣ dim_loan_products - Grade Distribution:")
grade_dist = pd.read_sql_query("""
    SELECT 
        grade,
        COUNT(*) as count,
        ROUND(AVG(loan_amount), 0) as avg_loan_amount,
        ROUND(AVG(interest_rate), 2) as avg_rate
    FROM dim_loan_products
    GROUP BY grade
    ORDER BY grade;
""", conn)
print(grade_dist)

# dim_geography - топ-10 штатов
print("\n3️⃣ dim_geography - Top 10 States:")
state_dist = pd.read_sql_query("""
    SELECT 
        state,
        COUNT(*) as loan_count
    FROM dim_geography
    GROUP BY state
    ORDER BY loan_count DESC
    LIMIT 10;
""", conn)
print(state_dist)

# fact_loan_performance - распределение outcomes
print("\n4️⃣ fact_loan_performance - Outcome Distribution:")
outcome_dist = pd.read_sql_query("""
    SELECT 
        outcome,
        COUNT(*) as count,
        ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (), 2) as percentage
    FROM fact_loan_performance
    GROUP BY outcome;
""", conn)
print(outcome_dist)

conn.close()

print("\n" + "="*80)
print("✅ DATABASE EXPLORATION COMPLETED!")
print("="*80)


👀 EXPLORING DATABASE TABLES

📊 TABLE 1: dim_borrowers (Информация о заемщиках)
--------------------------------------------------------------------------------
    loan_id  annual_income employment_length home_ownership  fico_low  \
0  68407277        55000.0         10+ years       MORTGAGE     675.0   
1  68355089        65000.0         10+ years       MORTGAGE     715.0   
2  68341763        63000.0         10+ years       MORTGAGE     695.0   
3  68476807       104433.0           3 years       MORTGAGE     695.0   
4  68426831        34000.0           4 years           RENT     690.0   
5  68476668       180000.0         10+ years       MORTGAGE     680.0   
6  67275481        85000.0         10+ years       MORTGAGE     705.0   
7  68466926        85000.0           6 years           RENT     685.0   
8  68616873        42000.0         10+ years       MORTGAGE     700.0   
9  68338832        64000.0           3 years       MORTGAGE     700.0   

   fico_high  debt_to_income  delin

In [7]:
import sqlite3
import pandas as pd

print("\n" + "="*80)
print("📊 SQL QUERY 2: COHORT ANALYSIS - Default Rate by Year")
print("="*80)

# Подключаюсь к базе данных
conn = sqlite3.connect('lending_club.db')

# COHORT ANALYSIS: Анализирую дефолты по годам выдачи кредитов
query2 = """
WITH cohort_data AS (
    -- Собираю данные по когортам (год выдачи кредита)
    SELECT 
        perf.issue_year as cohort_year,
        p.grade,
        perf.outcome,
        p.loan_amount,
        p.interest_rate,
        b.debt_to_income,
        b.fico_low
    FROM fact_loan_performance perf
    INNER JOIN dim_loan_products p ON perf.loan_id = p.loan_id
    INNER JOIN dim_borrowers b ON perf.loan_id = b.loan_id
    WHERE perf.outcome IN ('Good', 'Bad')
        AND perf.issue_year IS NOT NULL  -- исключаю строки без года
)
-- Группирую по году и рейтингу
SELECT 
    cohort_year,
    grade,
    COUNT(*) as total_loans,
    
    -- Считаю количество дефолтов
    SUM(CASE WHEN outcome = 'Bad' THEN 1 ELSE 0 END) as defaults,
    
    -- Считаю процент дефолтов
    ROUND(100.0 * SUM(CASE WHEN outcome = 'Bad' THEN 1 ELSE 0 END) / COUNT(*), 2) 
        as default_rate,
    
    -- Средние характеристики когорты
    ROUND(AVG(loan_amount), 0) as avg_loan_amount,
    ROUND(AVG(interest_rate), 2) as avg_interest_rate,
    ROUND(AVG(debt_to_income), 2) as avg_dti,
    ROUND(AVG(fico_low), 0) as avg_fico
    
FROM cohort_data
GROUP BY cohort_year, grade
ORDER BY cohort_year, grade;
"""

# Выполняю запрос
result2 = pd.read_sql_query(query2, conn)

print("\n🎯 Results: Cohort Analysis (First 20 rows)")
print("-"*80)
print(result2.head(20).to_string(index=False))

# Статистика по годам (без разбивки по grade)
print("\n\n📈 SUMMARY: Default Rate by Year (All Grades)")
print("-"*80)

query2_summary = """
SELECT 
    perf.issue_year as year,
    COUNT(*) as total_loans,
    ROUND(100.0 * SUM(CASE WHEN perf.outcome = 'Bad' THEN 1 ELSE 0 END) / COUNT(*), 2) 
        as default_rate,
    ROUND(AVG(p.loan_amount), 0) as avg_loan_amount
FROM fact_loan_performance perf
INNER JOIN dim_loan_products p ON perf.loan_id = p.loan_id
WHERE perf.outcome IN ('Good', 'Bad')
    AND perf.issue_year IS NOT NULL
GROUP BY perf.issue_year
ORDER BY perf.issue_year;
"""

summary = pd.read_sql_query(query2_summary, conn)
print(summary.to_string(index=False))

print("\n💡 Key Insights:")
# Нахожу год с наибольшим дефолтом
max_default_year = summary.loc[summary['default_rate'].idxmax()]
print(f"   • Highest default rate: {max_default_year['year']} ({max_default_year['default_rate']:.2f}%)")

# Нахожу год с наименьшим дефолтом
min_default_year = summary.loc[summary['default_rate'].idxmin()]
print(f"   • Lowest default rate: {min_default_year['year']} ({min_default_year['default_rate']:.2f}%)")

# Тренд за последние годы
recent_years = summary[summary['year'] >= 2015]
print(f"   • Recent years (2015-2018) avg default: {recent_years['default_rate'].mean():.2f}%")

conn.close()

print("\n" + "="*80)
print("✅ QUERY 2 COMPLETED!")
print("="*80)


📊 SQL QUERY 2: COHORT ANALYSIS - Default Rate by Year

🎯 Results: Cohort Analysis (First 20 rows)
--------------------------------------------------------------------------------
 cohort_year grade  total_loans  defaults  default_rate  avg_loan_amount  avg_interest_rate  avg_dti  avg_fico
      2007.0     A           74         4          5.41           4946.0               7.75     4.72     767.0
      2007.0     B           98        16         16.33           8338.0               9.42     8.65     719.0
      2007.0     C          141        34         24.11           8276.0              10.85    10.90     691.0
      2007.0     D           99        31         31.31           8006.0              12.55    11.69     669.0
      2007.0     E          100        31         31.00           8138.0              14.13    12.32     653.0
      2007.0     F           52        25         48.08          11851.0              15.58    13.53     655.0
      2007.0     G           35        17  

In [1]:
import sqlite3
import pandas as pd

print("\n" + "="*80)
print("📊 SQL QUERY 3: PORTFOLIO PROFITABILITY ANALYSIS")
print("="*80)

# Подключаюсь к базе данных
conn = sqlite3.connect('lending_club.db')

# PORTFOLIO PROFITABILITY: Анализирую прибыльность сегментов с учетом полного срока кредита
query3 = """
WITH loan_economics AS (
    -- Рассчитываю экономику каждого кредита с учетом полного срока
    SELECT 
        p.grade,
        p.term,
        p.loan_amount,
        p.interest_rate,
        perf.outcome,
        
        -- Определяю срок кредита в годах
        CASE 
            WHEN p.term = ' 36 months' THEN 3.0
            WHEN p.term = ' 60 months' THEN 5.0
        END as loan_term_years,
        
        -- Упрощенный расчет total interest за весь срок
        -- (линейное приближение для аналитических целей)
        p.loan_amount * (p.interest_rate / 100.0) * 
            CASE 
                WHEN p.term = ' 36 months' THEN 3.0
                WHEN p.term = ' 60 months' THEN 5.0
            END as total_interest_potential,
        
        -- Фактический доход (только если кредит вернули)
        CASE 
            WHEN perf.outcome = 'Good' THEN 
                p.loan_amount * (p.interest_rate / 100.0) * 
                    CASE 
                        WHEN p.term = ' 36 months' THEN 3.0
                        WHEN p.term = ' 60 months' THEN 5.0
                    END
            ELSE 0
        END as actual_revenue,
        
        -- Фактические потери (полная сумма кредита при дефолте)
        CASE 
            WHEN perf.outcome = 'Bad' THEN p.loan_amount
            ELSE 0
        END as actual_loss
        
    FROM fact_loan_performance perf
    INNER JOIN dim_loan_products p ON perf.loan_id = p.loan_id
    WHERE perf.outcome IN ('Good', 'Bad')
),
segment_metrics AS (
    -- Агрегирую метрики по сегментам
    SELECT 
        grade,
        term,
        COUNT(*) as loan_count,
        
        -- Показатели performance
        ROUND(100.0 * SUM(CASE WHEN outcome = 'Good' THEN 1 ELSE 0 END) / COUNT(*), 2) 
            as success_rate_pct,
        ROUND(100.0 * SUM(CASE WHEN outcome = 'Bad' THEN 1 ELSE 0 END) / COUNT(*), 2) 
            as default_rate_pct,
        
        -- Средние характеристики
        ROUND(AVG(loan_amount), 0) as avg_loan_amount,
        ROUND(AVG(interest_rate), 2) as avg_interest_rate,
        AVG(loan_term_years) as avg_term_years,
        
        -- Финансовые метрики (per loan average)
        ROUND(AVG(actual_revenue), 0) as avg_revenue_per_loan,
        ROUND(AVG(actual_loss), 0) as avg_loss_per_loan,
        ROUND(AVG(actual_revenue - actual_loss), 0) as avg_profit_per_loan,
        
        -- ROI calculation
        ROUND(100.0 * AVG(actual_revenue - actual_loss) / AVG(loan_amount), 2) 
            as roi_percent,
        
        -- Portfolio level metrics
        ROUND(SUM(actual_revenue) / 1000000.0, 2) as total_revenue_mm,
        ROUND(SUM(actual_loss) / 1000000.0, 2) as total_loss_mm,
        ROUND(SUM(actual_revenue - actual_loss) / 1000000.0, 2) as total_profit_mm
        
    FROM loan_economics
    GROUP BY grade, term
),
segment_ranking AS (
    -- Добавляю ранкинги и квартили для стратификации
    SELECT 
        *,
        
        -- Ранжирую по абсолютной прибыльности
        RANK() OVER (ORDER BY total_profit_mm DESC) as profit_rank,
        
        -- Ранжирую по эффективности (ROI)
        RANK() OVER (ORDER BY roi_percent DESC) as roi_rank,
        
        -- Ранжирую по объему
        RANK() OVER (ORDER BY loan_count DESC) as volume_rank,
        
        -- Квартили по объему для понимания концентрации портфеля
        NTILE(4) OVER (ORDER BY loan_count DESC) as volume_quartile
        
    FROM segment_metrics
)
-- Финальный output с аналитической интерпретацией
SELECT 
    grade,
    term,
    loan_count,
    default_rate_pct,
    avg_loan_amount,
    avg_interest_rate,
    avg_term_years,
    avg_profit_per_loan,
    roi_percent,
    total_profit_mm,
    profit_rank,
    roi_rank,
    volume_quartile,
    
    -- Стратегическая категоризация 
    CASE 
        -- High performers: высокая прибыль И хороший ROI
        WHEN roi_percent >= 40 AND total_profit_mm >= 100 THEN 'Star - Scale Aggressively'
        
        -- Cash cows: большой объем, стабильная прибыль
        WHEN volume_quartile = 1 AND roi_percent >= 20 THEN 'Core - Maintain & Optimize'
        
        -- Question marks: хороший ROI но малый объем
        WHEN roi_percent >= 40 AND loan_count < 50000 THEN 'Growth - Expand Carefully'
        
        -- Acceptable: прибыльные но не выдающиеся
        WHEN roi_percent >= 15 AND roi_percent < 40 THEN 'Acceptable - Monitor'
        
        -- Underperformers: низкая прибыльность
        WHEN roi_percent < 15 THEN 'Review - Reassess Strategy'
        
        ELSE 'Review'
    END as strategic_category
    
FROM segment_ranking
ORDER BY roi_percent DESC, total_profit_mm DESC;
"""

# Выполняю запрос
result3 = pd.read_sql_query(query3, conn)

print("\n Portfolio Profitability by Segment (Sorted by ROI)")
print("-"*80)
print(result3.to_string(index=False))

# === ANALYTICAL INSIGHTS ===
print("\n\n" + "="*80)
print(" EXECUTIVE SUMMARY")
print("="*80)

total_portfolio_profit = result3['total_profit_mm'].sum()
total_portfolio_volume = result3['loan_count'].sum()
weighted_avg_roi = (result3['avg_profit_per_loan'] * result3['loan_count']).sum() / result3['loan_count'].sum()
weighted_avg_roi_pct = (weighted_avg_roi / result3['avg_loan_amount'].mean()) * 100

print(f"\nPortfolio Overview:")
print(f"   • Total Segments Analyzed: {len(result3)}")
print(f"   • Total Loan Volume: {total_portfolio_volume:,}")
print(f"   • Total Portfolio Profit: ${total_portfolio_profit:,.1f}M")
print(f"   • Portfolio-Weighted Avg Profit/Loan: ${weighted_avg_roi:,.0f}")
print(f"   • Portfolio-Weighted ROI: {weighted_avg_roi_pct:.1f}%")

# Segment Distribution
print(f"\n Strategic Segmentation:")
strategy_dist = result3['strategic_category'].value_counts()
for category, count in strategy_dist.items():
    pct = (count / len(result3)) * 100
    print(f"   • {category}: {count} segments ({pct:.0f}%)")

# Top Performers
print(f"\n Top 3 Performers (by ROI):")
top3 = result3.nsmallest(3, 'roi_rank')
for idx, row in top3.iterrows():
    print(f"   {row['roi_rank']}. Grade {row['grade']}, {row['term'].strip()}")
    print(f"      ROI: {row['roi_percent']:.1f}% | Profit/Loan: ${row['avg_profit_per_loan']:,} | Volume: {row['loan_count']:,}")
    print(f"      Default Rate: {row['default_rate_pct']:.1f}% | Avg Rate: {row['avg_interest_rate']:.2f}%")

# Portfolio Concentration
print(f"\n Portfolio Concentration Analysis:")
q1_segments = result3[result3['volume_quartile'] == 1]
q1_volume_pct = (q1_segments['loan_count'].sum() / total_portfolio_volume) * 100
q1_profit_pct = (q1_segments['total_profit_mm'].sum() / total_portfolio_profit) * 100
print(f"   • Top Quartile (Q1): {len(q1_segments)} segments")
print(f"   • Q1 Volume Share: {q1_volume_pct:.1f}%")
print(f"   • Q1 Profit Contribution: {q1_profit_pct:.1f}%")

# Risk-Return Trade-off
print(f"\n Risk-Return Observation:")
high_risk = result3[result3['default_rate_pct'] >= 40]
if len(high_risk) > 0:
    avg_roi_high_risk = high_risk['roi_percent'].mean()
    print(f"   • Segments with 40%+ default rate: {len(high_risk)}")
    print(f"   • Their average ROI: {avg_roi_high_risk:.1f}%")
    print(f"   • Interpretation: High rates ({high_risk['avg_interest_rate'].mean():.1f}% avg) compensate for elevated risk")

# Term Analysis
print(f"\n Term Structure Analysis:")
term_comparison = result3.groupby('term').agg({
    'loan_count': 'sum',
    'roi_percent': 'mean',
    'total_profit_mm': 'sum'
}).round(2)
print(term_comparison)

conn.close()

print("\n" + "="*80)
print("✅ ANALYSIS COMPLETED")
print("="*80)


📊 SQL QUERY 3: PORTFOLIO PROFITABILITY ANALYSIS

 Portfolio Profitability by Segment (Sorted by ROI)
--------------------------------------------------------------------------------
grade       term  loan_count  default_rate_pct  avg_loan_amount  avg_interest_rate  avg_term_years  avg_profit_per_loan  roi_percent  total_profit_mm  profit_rank  roi_rank  volume_quartile         strategic_category
    B  60 months       47808             18.02          20394.0              10.70             5.0               5258.0        25.78           251.40            6         1                2       Acceptable - Monitor
    C  60 months      105553             27.53          19807.0              14.27             5.0               4925.0        24.87           519.88            2         2                2       Acceptable - Monitor
    A  60 months        6289             10.61          19299.0               7.87             5.0               4665.0        24.17            29.34           12    

In [15]:
import sqlite3
import pandas as pd

print("📤 Создаю Excel файл...")

conn = sqlite3.connect('lending_club.db')

# Query 1: Portfolio Profitability
query1 = """
WITH loan_economics AS (
    SELECT 
        p.grade,
        p.term,
        p.loan_amount,
        p.interest_rate,
        perf.outcome,
        
        CASE WHEN p.term = ' 36 months' THEN 3.0 ELSE 5.0 END as loan_term_years,
        
        CASE 
            WHEN perf.outcome = 'Good' THEN 
                p.loan_amount * (p.interest_rate / 100.0) * 
                    CASE WHEN p.term = ' 36 months' THEN 3.0 ELSE 5.0 END
            ELSE 0
        END as revenue,
        
        CASE WHEN perf.outcome = 'Bad' THEN p.loan_amount ELSE 0 END as loss
        
    FROM fact_loan_performance perf
    INNER JOIN dim_loan_products p ON perf.loan_id = p.loan_id
    WHERE perf.outcome IN ('Good', 'Bad')
)
SELECT 
    grade,
    term,
    COUNT(*) as loan_count,
    ROUND(100.0 * SUM(CASE WHEN outcome = 'Good' THEN 1 ELSE 0 END) / COUNT(*), 2) as success_rate,
    ROUND(100.0 * SUM(CASE WHEN outcome = 'Bad' THEN 1 ELSE 0 END) / COUNT(*), 2) as default_rate,
    ROUND(AVG(loan_amount), 0) as avg_loan_amount,
    ROUND(AVG(interest_rate), 2) as avg_interest_rate,
    ROUND(AVG(revenue - loss), 0) as avg_profit_per_loan,
    ROUND(100.0 * AVG(revenue - loss) / AVG(loan_amount), 2) as roi_percent,
    ROUND(SUM(revenue - loss) / 1000000.0, 2) as total_profit_mm
FROM loan_economics
GROUP BY grade, term;
"""

df1 = pd.read_sql_query(query1, conn)

# Query 2: Risk Scoring
query2 = """
WITH risk_factors AS (
    SELECT 
        perf.outcome,
        CASE 
            WHEN p.grade IN ('A', 'B') THEN 1
            WHEN p.grade IN ('C', 'D') THEN 2
            WHEN p.grade IN ('E', 'F') THEN 3
            ELSE 4
        END as grade_risk_score,
        CASE 
            WHEN b.debt_to_income < 10 THEN 1
            WHEN b.debt_to_income < 20 THEN 2
            WHEN b.debt_to_income < 30 THEN 3
            ELSE 4
        END as dti_risk_score,
        CASE 
            WHEN b.fico_low >= 750 THEN 1
            WHEN b.fico_low >= 700 THEN 2
            WHEN b.fico_low >= 650 THEN 3
            ELSE 4
        END as fico_risk_score,
        CASE WHEN p.term = ' 36 months' THEN 1 ELSE 2 END as term_risk_score
    FROM fact_loan_performance perf
    INNER JOIN dim_loan_products p ON perf.loan_id = p.loan_id
    INNER JOIN dim_borrowers b ON perf.loan_id = b.loan_id
    WHERE perf.outcome IN ('Good', 'Bad')
)
SELECT 
    (grade_risk_score + dti_risk_score + fico_risk_score + term_risk_score) as composite_risk_score,
    COUNT(*) as loan_count,
    ROUND(100.0 * SUM(CASE WHEN outcome = 'Bad' THEN 1 ELSE 0 END) / COUNT(*), 2) as default_rate
FROM risk_factors
GROUP BY composite_risk_score
ORDER BY composite_risk_score;
"""

df2 = pd.read_sql_query(query2, conn)

# Query 3: Cohort Analysis  
query3 = """
SELECT 
    perf.issue_year as cohort_year,
    p.grade,
    COUNT(*) as total_loans,
    ROUND(100.0 * SUM(CASE WHEN perf.outcome = 'Bad' THEN 1 ELSE 0 END) / COUNT(*), 2) as default_rate,
    ROUND(AVG(p.loan_amount), 0) as avg_loan_amount
FROM fact_loan_performance perf
INNER JOIN dim_loan_products p ON perf.loan_id = p.loan_id
WHERE perf.outcome IN ('Good', 'Bad') AND perf.issue_year IS NOT NULL
GROUP BY perf.issue_year, p.grade;
"""

df3 = pd.read_sql_query(query3, conn)
conn.close()

# Создаём Excel с 3 листами
with pd.ExcelWriter('lending_club_data.xlsx', engine='openpyxl') as writer:
    df1.to_excel(writer, sheet_name='Portfolio', index=False)
    df2.to_excel(writer, sheet_name='Risk', index=False)
    df3.to_excel(writer, sheet_name='Cohort', index=False)

print("\n✅ ФАЙЛ СОЗДАН: lending_club_data.xlsx")
print("\nЛисты в файле:")
print("  1. Portfolio (14 строк)")
print("  2. Risk (10 строк)")
print("  3. Cohort (84 строки)")
print("\nТеперь импортируй этот файл в Power BI!")

📤 Создаю Excel файл...

✅ ФАЙЛ СОЗДАН: lending_club_data.xlsx

Листы в файле:
  1. Portfolio (14 строк)
  2. Risk (10 строк)
  3. Cohort (84 строки)

Теперь импортируй этот файл в Power BI!
